In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
pizza_df = pd.read_csv("pizza_data.csv")

In [9]:
api_key = ""
def get_google_results(address, api_key=None, return_full_response=False):
    # Set up your Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [6]:
pizza_df['address_total'] = pizza_df['address1'] + ", " + pizza_df['city'] + "," + pizza_df['country']

In [ ]:
results = []
for address in pizza_df['address_total']:
    geocoded = False
    while geocoded is not True:
        try:
            geocode_result = get_google_results(address, api_key, return_full_response=True)
        except Exception as e:
            print(e)
            print("Major error with {}".format(address))
            print("Skipping!")
            geocoded = True
            
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            print("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            if geocode_result['status'] != 'OK':
                print("Error geocoding {}: {}".format(address, geocode_result['status']))
            print("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

In [ ]:
pizza_df['address_total'][0]

In [24]:
pizza_df = pd.concat([pizza_df, json_normalize(results)], axis=1)

In [25]:
pizza_df.to_csv("pizza_data.csv")